In [ ]:
from django.db.models import Q
from aumentos.models import Proyecto, Movimiento, Aumento
from hiscar.models import Hiscar
from django.contrib.auth.models import User
from django.utils.timezone import now


In [ ]:
try:
    import sqlparse
except:
    !pip install sqlparse
    import sqlparse

In [ ]:
user = User.objects.get(id=1)
#Todos los proyetos
ps = Proyecto.objects.all()
ps_ids = [p.id for p in ps]

#Todos los movimientos de esos proyectos
ms = Movimiento.objects.filter(proyecto_id__in=ps_ids).order_by('orden')
ms_ids = [m.id for m in ms]

#Todos los aumentos de esos movimientos
as_ = Aumento.objects.filter(movimiento_id__in=ms_ids).order_by('orden')

as_ids = [a.id for a in as_]
Hiscar.objects.filter(aumento_id__in=as_ids).delete()
#Los periodos y reparticion de esos movimientos
prs = [(m.periodo_dd, m.periodo_ht, m.reparticion.codigo) for m in ms]

#Hiscars de las reparticiones en los periodos de los movimientos
q = Q()
for pr in prs:
    q.add(Q(periodo__gte=pr[0], periodo__lte=pr[1], reparticion=pr[2]), Q.OR)
h = Hiscar.objects.\
    filter(
        q,
        archivo__isnull=False).\
    order_by(
        'periodo',
        'reparticion',
        'cargo').\
    all()
# FIXME Aplciar solo el ultimo y hacer un for 
# Aplicar aumentos
for m in ms:
    fh = int(m.proyecto.fecha_hiscar.strftime ("%Y%m"))
    print(m.descripcion, m.periodo_dd, m.periodo_ht)
    hiscars = h.filter(periodo=m.periodo_dd)
    for aumento in as_.filter(movimiento=m):
        # Hiscars con las que voy a trabajar
        hiscar = hiscars.filter(aumento._get_hiscars_filter())
        # Prints de las querys para debuggear
        print('aumento modo ' ,aumento.modo_aumento())
        print('-'*30)
        query_str = sqlparse.split(str(hiscar.query))
        for st in query_str:
            print(sqlparse.format(st, reindent=True, keyword_case='upper'))
        print('-'*30)
        hiscar_aumentos = []
        #Aplico aumentos a cada hiscar.
        for hisc in hiscar:
            sueldo = float(getattr(hisc, aumento.tipo_aumento.nombre))
            if aumento.tipo_calculo == 'Porcentaje':
                monto_aumento = sueldo * float(aumento.valor) / 100
            else:
                monto_aumento = float(aumento.valor)
            nuevo_sueldo = sueldo + monto_aumento
            setattr(hisc, aumento.tipo_aumento.nombre, nuevo_sueldo)
            
            hisc.pk = None
            hisc.archivo = None
            hisc.periodo = 0
            hisc.aumento = aumento
            hiscar_aumentos.append(hisc)
            print(hisc.periodo, hisc.reparticion + hisc.cargo,'$', sueldo,  '===> $', nuevo_sueldo, '+ $', monto_aumento)
        Hiscar.objects.bulk_create(hiscar_aumentos)
